In [12]:
import pandas as pd 


In [20]:
df = {}
import os 
for i in range(8):  
    # model = "ETTh2"
    # model = "ETTm1"
    # model = "exchange_rate"
    model = "weather" 

    tmp = pd.read_csv(os.path.join('./mainresult/', str("PatchTST"), model + str(i) + "df.csv"))
    df[i] = tmp


In [8]:
df[7]
# get mean mse and mae for re=0 in df[0]


,Unnamed: 0,len,re,mse,mae
0,0,96.0,0.0,0.306364,0.362063
1,1,96.0,1.0,0.304745,0.357949
2,2,192.0,0.0,0.345471,0.384323
3,3,192.0,1.0,0.343330,0.376751
4,4,336.0,0.0,0.377882,0.402008
5,5,336.0,1.0,0.373807,0.395046
6,6,720.0,0.0,0.427765,0.432952
7,7,720.0,1.0,0.423601,0.422774


In [21]:
for i in range(8): 
    dot = "" 
    if i == 1 or i == 2  or i == 4 or i == 7: 
        dot = '*'
    print(str(i) + dot + "\n", df[i][df[i]["re"]==0].mean().round(3)[-1], df[i][df[i]["re"]==0].mean().round(3)[-2]," ", df[i][df[i]["re"]==1].mean().round(3)[-1], df[i][df[i]["re"]==1].mean().round(3)[-2])

0
 0.264 0.23   0.26 0.23
1*
 0.296 0.253   0.296 0.251
2*
 0.292 0.261   0.282 0.249
3
 0.282 0.241   0.273 0.235
4*
 0.322 0.275   0.304 0.261
5
 0.285 0.24   0.283 0.241
6
 0.28 0.24   0.275 0.238
7*
 0.28 0.235   0.273 0.234


In [34]:
"params : "
import re

def process_params(params_str):
    # 使用正则表达式来提取字典参数
    params_str = params_str[:-37] + "}"
    param_dict = {}
    def device(type='w', index=1):
        return 1
    # print(params_str)
    # print(eval(params_str))
    try:
        param_dict = eval(params_str)
    except Exception as e:
        print(f"Error parsing parameters: {e}")
    return param_dict

# 定义一个函数f，用于处理参数
def f(params_dict):
    # 在这里实现你的处理逻辑
    print("Received parameters:")
    for key, value in params_dict.items():
        print(f"{key}: {value}")

# 读取文件并处理参数
with open('abltest.log', 'r') as file:
    content = file.read()

# 使用正则表达式查找 "params : " 标识符，并提取参数
params_matches = re.finditer(r'params\s*:\s*({[^}]*})', content)

for match in params_matches:
    params_str = match.group(1)
    print(match)
    params_dict = process_params(params_str)
    # print(params_str)

<re.Match object; span=(124, 2735), match="params : {'loss': 'huber', 'conf': 'ECL-PatchTST'>
<re.Match object; span=(5128, 7733), match="params : {'loss': 'huber', 'conf': 'ECL-PatchTST'>
<re.Match object; span=(10124, 12728), match="params : {'loss': 'huber', 'conf': 'ECL-PatchTST'>
<re.Match object; span=(14103, 16708), match="params : {'loss': 'huber', 'conf': 'ECL-PatchTST'>
<re.Match object; span=(18083, 20684), match="params : {'loss': 'mse', 'conf': 'ECL-PatchTST', >
<re.Match object; span=(22058, 24671), match="params : {'loss': 'huber', 'conf': 'ECL-PatchTST'>
<re.Match object; span=(27080, 29687), match="params : {'loss': 'huber', 'conf': 'ECL-PatchTST'>
<re.Match object; span=(32095, 34701), match="params : {'loss': 'huber', 'conf': 'ECL-PatchTST'>
<re.Match object; span=(36085, 38692), match="params : {'loss': 'huber', 'conf': 'ECL-PatchTST'>
<re.Match object; span=(40075, 42678), match="params : {'loss': 'mse', 'conf': 'ECL-PatchTST', >
<re.Match object; span=(44060, 4667